In [1]:
# %pip install ipympl

In [2]:
%matplotlib widget
import numpy as np
import caiman as cm
import matplotlib.pyplot as plt
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf
from caiman.source_extraction.cnmf.params import CNMFParams
from caiman.utils.visualization import view_quilt
import pandas as pd
import sciebo

In [3]:
sciebo.download_file_from_sciebo('https://uni-bonn.sciebo.de/s/RR7qj7tklW1rX25', 'data', 'Sue_2x_3000_40_-46.tif')
sciebo.download_file_from_sciebo('https://uni-bonn.sciebo.de/s/RR7qj7tklW1rX25', 'data', 'data_endoscope.tif')

In [4]:
fname = "data/Sue_2x_3000_40_-46.tif"
movie_orig = cm.load(fname)

In [5]:
# motion correction
params = CNMFParams()
motion_params = {
    'strides': (48, 48),
    'overlaps': (24, 24),
    'max_shifts': (6, 6),
    'max_deviation_rigid': 3,
    'pw_rigid': True
}
params.motion.update(motion_params)
mc = MotionCorrect(fname, **params.motion)
mc.motion_correct(save_movie=True)
mc.fname_tot_els

['data/Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap']

In [6]:
mc_fname = cm.save_memmap(
    mc.fname_tot_els,
    base_name='memmap_',
    order='C'
)

In [7]:
Yr, dims, num_frames = cm.load_memmap(mc_fname)
images = np.reshape(Yr.T, [num_frames] + list(dims), order='F')
images.shape

(3000, 170, 170)

In [8]:
patch_params = {
    "rf": 15,
    "stride": 10,
    'gSig': 4,
    'K': 7    
}
params.patch.update(patch_params)
cnmf_model = cnmf.CNMF(n_processes=1, params=params)

In [9]:
cnmf_fit = cnmf_model.fit(images)

In [ ]:
cnmf_refit = cnmf_fit.refit(images)

In [ ]:
movie_corrected = cm.load(mc.mmap_file) # load motion corrected movie
correlation_image = cm.local_correlations(movie_corrected, swap_dim=False)

In [ ]:
print("Thresholds to be used for evaluate_components()")
print(f"min_SNR = {cnmf_refit.params.quality['min_SNR']}")
print(f"rval_thr = {cnmf_refit.params.quality['rval_thr']}")
print(f"min_cnn_thr = {cnmf_refit.params.quality['min_cnn_thr']}")

In [ ]:
cnmf_refit.estimates.evaluate_components(images, cnmf_refit.params);


In [ ]:
print(f"Num accepted/rejected: {len(cnmf_refit.estimates.idx_components)}, {len(cnmf_refit.estimates.idx_components_bad)}")

In [ ]:
cnmf_refit.estimates.plot_contours_nb(img=correlation_image, 
                                      idx=cnmf_refit.estimates.idx_components);

In [ ]:
cnmf_refit.estimates.nb_view_components(img=correlation_image, 
                                        idx=cnmf_refit.estimates.idx_components,
                                        cmap='gray',
                                        denoised_color='red');

In [ ]:
cnmf_refit.estimates.nb_view_components(img=correlation_image, 
                                        idx=cnmf_refit.estimates.idx_components_bad, 
                                        cmap='gray',
                                        denoised_color='red')

delta F/F

In [ ]:
cnmf_refit.estimates.detrend_df_f(quantileMin=8, 
                                    frames_window=250,
                                    flag_auto=False,
                                    use_residuals=False); 

In [ ]:
cnmf_refit.estimates.nb_view_components(img=correlation_image, 
                                        idx=cnmf_refit.estimates.idx_components,
                                        cmap='gray',
                                        denoised_color='red');

In [ ]:
frame_rate = cnmf_refit.params.data['fr']
frame_pd = 1/frame_rate
frame_times = np.linspace(0, num_frames*frame_pd, num_frames);

In [ ]:
idx_to_plot = 30
idx_accepted = cnmf_refit.estimates.idx_components
component_number = idx_accepted[idx_to_plot]
f, ax = plt.subplots(figsize=(7,2))
ax.plot(frame_times, 
        cnmf_refit.estimates.F_dff[component_number, :], 
        linewidth=0.5,
        color='k');
ax.set_xlabel('Time (s)')
ax.set_ylabel('$\Delta F/F$')
ax.set_title(f"$\Delta F/F$ for unit {component_number}");
plt.tight_layout()